# 通过房价预测解释数据分布的变化和灾难性遗忘


在过去的几年里，许多论文已经表明，当随着时间的推移不断学习并且无法访问以前遇到的数据时，遗忘对神经网络的影响。 然而，用于描述该现象的许多示例都相当复杂，涉及数以千计甚至数百万个参数。

在这个简短的笔记本中，我将尝试制作神经网络中灾难性遗忘的最简单示例，仅使用**一个神经元**和**两个参数**（权重和偏差项），即使用线性回归 .

我们将建立在“*房价*”数据集和 Andrew Ng 著名的 Coursera“*机器学习*”课程中使用的示例之上，我们将：

1. 建立持续学习环境
2. 为线性回归模型显示理想的训练参数
3. 显示改变数据分布时遗忘的影响



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
import unittest

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8
rcParams['animation.embed_limit'] = 2**128

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
# !wget https://raw.githubusercontent.com/Data-Science-FMI/ml-from-scratch-2019/master/data/house_prices_train.csv

这是我们将要使用的数据集的摘要以及他的一些主要属性：

In [ ]:
df_train = pd.read_csv('house_prices_train.csv')
df_train.describe()

In [ ]:
df_train['SalePrice'].describe()

In [ ]:
sns.distplot(df_train['SalePrice']);

Below we can see how the Living Room square feets nicely correlates with the House sale price:

In [ ]:
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000), s=32);

现在，为了创建一个持续学习设置，我们将数据集分成两部分，两个不同的批次：
 - 第一个在**2000 年之前**建造的房屋中，
 - 第二个在**2000 年之后**的房屋中。

In [ ]:
df_new = df_train[df_train.YearBuilt > 2000]
df_old = df_train[df_train.YearBuilt <= 2000]

In [ ]:
df_new.describe()

In [ ]:
df_old.describe()

In [ ]:
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_new[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000), s=32, color="orange");

In [ ]:
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_old[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000), s=32, color="blue");

平均而言，最近的房子以更高的价格出售是有道理的。**当然新房子的整体价格会比较高一些**


下面我们在整个训练集上训练我们的单个神经元的线性回归模型以获得最佳参数：

In [ ]:
cumul_x = df_train['GrLivArea']
cumul_y = df_train['SalePrice']

# x = (x - x.mean()) / x.std()
cumul_x = np.c_[np.ones(cumul_x.shape[0]), cumul_x] 

cumul_x.shape

In [ ]:
def loss(h, y):
  sq_error = (h - y)**2
  n = len(y)
  return 1.0 / (2*n) * sq_error.sum()

In [ ]:
class LinearRegression:

  def __init__(self):
    
    self._W = np.zeros(2)
    self._cost_history = []
    self._w_history = [self._W]
  
  def predict(self, X):

    return np.dot(X, self._W)
  
  def _gradient_descent_step(self, X, targets, lr):

    predictions = self.predict(X)
    
    error = predictions - targets
    gradient = np.dot(X.T,  error) / len(X)

    self._W -= lr * gradient
      
  def fit(self, X, y, n_iter=100000, lr=0.01):

    for i in range(n_iter):
      
        prediction = self.predict(X)
        cost = loss(prediction, y)
        
        self._cost_history.append(cost)
        
        self._gradient_descent_step(X, y, lr)
        
        self._w_history.append(self._W.copy())
        
    return self
      
        

In [ ]:
cumul_clf = LinearRegression()
cumul_clf.fit(cumul_x, cumul_y, n_iter=150, lr=1e-7)

cumul_clf._W

In [ ]:
plt.title('Cost Function J')
plt.xlabel('No. of iterations')
plt.ylabel('Cost')
plt.plot(cumul_clf._cost_history)
plt.show()

In [ ]:
#Animation
def animate(clf, set_x, set_y, frames=150):
    #Set the plot up,
    fig = plt.figure()
    ax = plt.axes()
    plt.title('Sale Price vs Living Area')
    plt.xlabel('Living Area in square feet')
    plt.ylabel('Sale Price ($)')
    if len(set_x) == 1:
        plt.scatter(set_x[0][:,1], set_y[0])
    else:
        plt.scatter(set_x[0][:,1], set_y[0], color="blue")
        plt.scatter(set_x[1][:,1], set_y[1], color="orange")
    line, = ax.plot([], [], lw=2, color='red')
    annotation = ax.text(200, 700000, '')
    # optimal
    x = np.linspace(0, 7000, 1000)
    y = cumul_clf._W[1]*x + cumul_clf._W[0]
    ax.plot(x, y, 'g--')
    annotation.set_animated(True)
    plt.close()

    #Generate the animation data,
    def init():
        line.set_data([], [])
        annotation.set_text('')
        return line, annotation

    # animation function.  This is called sequentially
    def animate(i):
        # x = np.linspace(-5, 20, 1000)
        x = np.linspace(0, 7000, 1000)
        y = clf._w_history[i][1]*x + clf._w_history[i][0]
        line.set_data(x, y)
        annotation.set_text(
            'Cost = %.2f e10\nWeight: %.2f\nBias: %.2f' % 
            (clf._cost_history[i]/1e10, clf._w_history[i][1],
             clf._w_history[i][0]))
        return line, annotation

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                frames=frames, interval=10, blit=True)

    rc('animation', html='jshtml')

    return anim

In [ ]:
anim = animate(cumul_clf, [cumul_x], [cumul_y])
anim

好的，所以这项工作的最佳参数是权重：9.94290254e-02 和偏差：1.18069042e+02。 这将在图中显示为绿色虚线。 现在让我们移动持续学习场景。

在这种情况下，我们将从第一批数据（即包含所有旧房屋数据的批次）开始，然后，使用在此步骤计算的最佳参数，我们将尝试对第二批的数据（使用 最新房屋数据）。

In [ ]:
x_old = df_old['GrLivArea']
y_old = df_old['SalePrice']

x_old = np.c_[np.ones(x_old.shape[0]), x_old] 

x_old.shape

In [ ]:
x_new = df_new['GrLivArea']
y_new = df_new['SalePrice']

x_new = np.c_[np.ones(x_new.shape[0]), x_new] 

x_new.shape

In [ ]:
cl_clf = LinearRegression()
cl_clf.fit(x_old, y_old, n_iter=150, lr=1e-7)

cl_clf._W

In [ ]:
anim = animate(cl_clf, [x_old, x_new], [y_old, y_new])
anim

在上图中，我们可以看到该模型仅符合我们预期的旧房数据！ 让我们知道如果我们在最新的房屋批次上微调模型会发生什么！

In [ ]:
# cl_clf = LinearRegression()
cl_clf.fit(x_new, y_new, n_iter=150, lr=1e-7)

cl_clf._W

In [ ]:
plt.title('Cost Function J')
plt.xlabel('No. of iterations')
plt.ylabel('Cost')
plt.plot(cl_clf._cost_history)
plt.show()

In [ ]:
anim = animate(cl_clf, [x_old, x_new], [y_old, y_new], frames=300)
anim

因此，我们从上图中可以看出，即使我们从上一步的最佳解决方案开始，我们的权重和偏差参数也会以某种方式被覆盖，只是为了适应最新房屋的新数据分布。

在这里，我们基本上“*忘记*”了如何正确预测 2000 年之前建造的房屋的价格，只是为了更好地预测 2000 年之后建造的房屋的价格，尽管（这里的重点）确实存在更好和通用的参数化** ** 并且会减少总预测误差。

我们如何在不访问先前遇到的数据的情况下有效地学习最佳参数化是持续学习的主要焦点之一。